<a href="https://colab.research.google.com/github/mc-friday/hanghaeAI/blob/main/%5B4%EC%A3%BC%EC%B0%A8%5D%EC%8B%AC%ED%99%94%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [4주차] 심화과제: 수능 국어 문제 GPT-4로 풀어보기

In [ ]:
!pip install openai

In [ ]:
import openai
import json
import time

openai.api_key = "key"
client = openai.OpenAI(api_key=openai.api_key)

## 1. 수능 문제 JSON 데이터 로드 함수

In [ ]:
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

## 2. 하나의 문제에 대해서 GPT-4의 예측 결과를 내놓는 함수

In [ ]:
def prediction(problem):
    """
    문제 데이터를 받아 GPT-4가 답안을 예측하도록 하는 함수
    """
    # And if there is content in More info, you must also refer to More info to choose an answer.
    # GPT-4에 전달할 프롬프트 템플릿 (일부러 영어를 사용함)
    zero_shot_cot_en_prompt_plus = f"""
    국어 시험 문제를 푸는 똑똑한 학생으로서 다음 문제의 답을 구하세요.
    지문을 읽고, 질문에 대한 답을 1부터 5까지의 선택지 중에 한 개만 골라서 대답해야 합니다.
    <보기>가 있으면 꼭 문제 풀이에 참고해야 합니다.

    지문 :
    {problem['paragraph']}

    질문 :
    {problem['question']}

    <보기> :
    {problem.get('question_plus', '')}

    선택지 :
    1 - {problem['choices'][0]}
    2 - {problem['choices'][1]}
    3 - {problem['choices'][2]}
    4 - {problem['choices'][3]}
    5 - {problem['choices'][4]}

    1번, 2번, 3번, 4번, 5번 중에 하나를 정답으로 고르세요.
    단계별로 생각하며 정답을 고르세요.
    정답은 숫자만 답하세요.
    정답:
    """

     # 문제 풀이 시작 시간 기록
    start_time = time.time()


    # GPT-4 API 호출
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": zero_shot_cot_en_prompt_plus}
            ]
        )
        # GPT-4의 답변에서 선택한 답안을 반환
        predicted_answer = response.choices[0].message.content

         # 문제 풀이 종료 시간 기록
        end_time = time.time()
        elapsed_time = end_time - start_time

        print(f"[LOG] 문제 풀이 시간: {elapsed_time:.2f}초, 예측된 정답: {predicted_answer}")

        return predicted_answer
    except Exception as e:
        print(f"Error during GPT-4 prediction: {e}")
        return None


## 3. 모든 문제를 평가하고 점수를 계산하는 함수

In [ ]:
def evaluate_all_problems(data):
    """
    JSON 데이터의 모든 문제를 평가하고 점수를 계산하는 함수
    """
    total_score = 0  # 총 점수
    possible_score = 0  # 가능한 총 점수
    correct_count = 0  # 정답 개수
    total_count = 0  # 전체 문제 수

    for item in data:
        paragraph = item["paragraph"]
        for problem in item["problems"]:
            total_count += 1
            possible_score += problem["score"]  # 가능한 점수 누적

            # 문제 데이터 준비
            problem_data = {
                "paragraph": paragraph,
                "question": problem["question"],
                "choices": problem["choices"],
                "question_plus": problem.get("question_plus", ""),
            }

            # GPT-4 답안 예측
            predicted_answer = prediction(problem_data)

            # 실제 정답과 비교
            if str(predicted_answer) == str(problem["answer"]):  # 정답 비교
                total_score += problem["score"]
                correct_count += 1

    # 결과 출력
    print(f"총 문제 수: {total_count}")
    print(f"정답 개수: {correct_count}/{total_count}")
    print(f"정확도: {correct_count / total_count:.2%}")
    print(f"총 점수: {total_score}/{possible_score}")
    return total_score, possible_score, correct_count, total_count

In [ ]:
file_path = "./data/2023_11_KICE.json"  # Colab에 업로드한 JSON 파일 경로

# 데이터 로드
data = load_data(file_path)

# 모든 문제 평가 실행
total_score, possible_score, correct_count, total_count = evaluate_all_problems(data)

[LOG] 문제 풀이 시간: 0.32초, 예측된 정답: 4
[LOG] 문제 풀이 시간: 0.29초, 예측된 정답: 5
[LOG] 문제 풀이 시간: 0.45초, 예측된 정답: 1
[LOG] 문제 풀이 시간: 0.48초, 예측된 정답: 4
[LOG] 문제 풀이 시간: 0.44초, 예측된 정답: 5
[LOG] 문제 풀이 시간: 0.43초, 예측된 정답: 3
[LOG] 문제 풀이 시간: 0.49초, 예측된 정답: 2
[LOG] 문제 풀이 시간: 0.42초, 예측된 정답: 3
[LOG] 문제 풀이 시간: 0.39초, 예측된 정답: 2
[LOG] 문제 풀이 시간: 0.52초, 예측된 정답: 1
[LOG] 문제 풀이 시간: 0.54초, 예측된 정답: 5
[LOG] 문제 풀이 시간: 0.91초, 예측된 정답: 2
[LOG] 문제 풀이 시간: 4.01초, 예측된 정답: 1

- ⓐ(맞는)에서는 문맥상 '적절한', '올바른'이라는 의미로 사용되고 있습니다. 선택지 1에서 "이것이 네가 찾는 자료가 ⓐ(맞는지) 확인해 보아라."는 올바른 자료인지 확인하라는 의미로 적절합니다.
- ⓑ, ⓒ, ⓓ, ⓔ는 각각 다음과 같은 의미로 사용됩니다:
  - ⓑ(들) 수 있다: "예로 들 수 있다."로 문맥상 사용되어서 나머지 선택지와는 다릅니다.
  - ⓒ(받을) 수 있고: '받을 수 있다'는 의미.
  - ⓓ(고를) 수 있는: '선택할 수 있는' 의미. 
  - ⓔ(지켜야) 하는: '준수해야 하는' 의미. 

따라서, 문맥상 ⓐ의 의미와 가장 가까운 것은 1번 선택지입니다.
[LOG] 문제 풀이 시간: 0.32초, 예측된 정답: 3
[LOG] 문제 풀이 시간: 0.27초, 예측된 정답: 5
[LOG] 문제 풀이 시간: 0.25초, 예측된 정답: 4
[LOG] 문제 풀이 시간: 0.31초, 예측된 정답: 1
[LOG] 문제 풀이 시간: 0.29초, 예측된 정답: 4
[LOG] 문제 풀이 시간: 0.44초, 예측된 정답: 3
[LOG] 문제 풀이 시간: 0.23초, 예측된 정답: 3
[LOG]